In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np


In [2]:
ol = Overlay("./design_1_demofp.bit")
ol.ip_dict

{'example_0': {'fullpath': 'example_0',
  'type': 'xilinx.com:hls:example:1.0',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1073741824,
  'mem_id': 's_axi_control',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_CONTROL_ADDR_WIDTH': '4',
   'C_S_AXI_CONTROL_DATA_WIDTH': '32',
   'Component_Name': 'design_1_example_0_0',
   'clk_period': '10',
   'machine': '64',
   'combinational': '0',
   'latency': '17',
   'II': 'x',
   'EDK_IPTYPE': 'PERIPHERAL',
   'C_S_AXI_CONTROL_BASEADDR': '0x40000000',
   'C_S_AXI_CONTROL_HIGHADDR': '0x4000FFFF'},
  'registers': {'CTRL': {'address_offset': 0,
    'size': 32,
    'access': 'read-write',
    'description': 'Control signals',
    'fields': {'AP_START': {'bit_offset': 0,
      'bit_width': 1,
      'description': 'Control signals',
      'access': 'read-write'},
     'AP_DONE': {'bit_offset': 1,
      'bit_width': 1,
      'description': 'Control signals',
      'access': 'read-onl

In [3]:
dma0 = ol.axi_dma_0
dma0_send = ol.axi_dma_0.sendchannel
dma0_recv = ol.axi_dma_0.recvchannel

dma1 = ol.axi_dma_1
dma1_send = ol.axi_dma_1.sendchannel

hls_ip = ol.example_0
hls_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [4]:
CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x1) # 0x81 will set bit 0
hls_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [5]:
NUM_SAMPLES = 10
input_buffer0 = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)
input_buffer1 = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)
output_buffer = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)

a = [i for i in range(NUM_SAMPLES)]
a1 = np.float32(a)
a2 = 1.0
np.copyto(input_buffer0, a1)
np.copyto(input_buffer1, a2)


In [6]:
for i in range(NUM_SAMPLES):
    print(input_buffer0[i], input_buffer1[i], output_buffer[i])

0.0 1.0 0.0
1.0 1.0 0.0
2.0 1.0 0.0
3.0 1.0 0.0
4.0 1.0 0.0
5.0 1.0 0.0
6.0 1.0 0.0
7.0 1.0 0.0
8.0 1.0 0.0
9.0 1.0 0.0


In [7]:
dma0_recv.transfer(output_buffer)
dma0_send.transfer(input_buffer0)
dma1_send.transfer(input_buffer1)
dma0_send.wait()
dma1_send.wait()
dma0_recv.wait()

In [9]:
dma0.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=377786368),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=40),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlv

In [10]:
dma1.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=377790464),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=40),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=0, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlv

In [11]:
# out = in1 + in2 + 5
for i in range(NUM_SAMPLES):
    if np.absolute(output_buffer[i] - input_buffer0[i] - input_buffer1[i]) < 0.001:
        print(input_buffer0[i], '+', input_buffer1[i],  'is roughly', output_buffer[i])
    else:
        print('error!', input_buffer0[i], '+', input_buffer1[i], 'should not be', output_buffer[i])

0.0 + 1.0 is roughly 1.0
1.0 + 1.0 is roughly 2.0
2.0 + 1.0 is roughly 3.0
3.0 + 1.0 is roughly 4.0
4.0 + 1.0 is roughly 5.0
5.0 + 1.0 is roughly 6.0
6.0 + 1.0 is roughly 7.0
7.0 + 1.0 is roughly 8.0
8.0 + 1.0 is roughly 9.0
9.0 + 1.0 is roughly 10.0


In [12]:
del input_buffer0
del input_buffer1
del output_buffer
del ol